## 模型设计

专用测试模型

由于GridSearchCV耗时过久，考虑折中的方法，即使用原始的train中的分割作为调参的评判

In [208]:
#!/usr/bin/env python
# -*- coding: UTF-8 -*-
import sys
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
import operator
from matplotlib import pylab as plt
from datetime import datetime
import time
from sklearn.model_selection import GridSearchCV

In [209]:
# import gc
def show_record():
    train = pd.read_csv('train_set.csv')
#     valid = pd.read_csv('val_set.csv')
#     label_val = pd.read_csv('label_val_set.csv')
    valid1 = pd.read_csv('val_1.csv')
    valid2 = pd.read_csv('val_2.csv')
    valid3 = pd.read_csv('val_3.csv')
#     test = pd.read_csv('test_set.csv')
    print train.shape
#     print valid.shape
#     print label_val.shape
#     print test.shape
    print valid1.shape
    print valid2.shape
    print valid3.shape

# show_record()
# del train, valid, test
# gc.collect()


### 训练数据
* 返回训练后的模型
* 生成特征map文件作为后续特征重要性之用

In [210]:
def create_feature_map(features):
    outfile = open(r'xgb.fmap', 'w')
    i = 0
    for feat in features:
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
        i = i + 1
    outfile.close()

def xgb_model(train_set):
    actions = pd.read_csv(train_set)             #read train_set
    # 单纯的删掉模型前一遍训练认为无用的特征（根据特征重要性中不存在的特征）
    lst_useless = ['brand']
    
    actions.drop(lst_useless, inplace=True, axis=1)
    
    users = actions[['user_id', 'sku_id']].copy()
    labels = actions['label'].copy()
    del actions['user_id']
    del actions['sku_id']
    del actions['label']
    # 尝试通过设置scale_pos_weight来调整政府比例不均的问题，但是经过采样的正负比为1:10，训练结果反而不如设置为1
#     ratio = float(np.sum(labels==0)) / np.sum(labels==1)
#     print ratio
    
    # write to feature map
    features = list(actions.columns[:])
    print 'total features: ', len(features)
    create_feature_map(features)
    # 训练时即传入特征名
#     features = list(actions.columns.values)
    
    user_index=users
    training_data=actions
    label=labels
    X_train, X_valid, y_train, y_valid = train_test_split(training_data.values, label.values, test_size=0.2, 
                                                          random_state=0)
    
    # 尝试通过提前设置传入训练的正负例的权重来改善正负比例不均的问题
#     weights = np.zeros(len(y_train))
#     weights[y_train==0] = 1
#     weights[y_train==1] = 10
    
#     dtrain = xgb.DMatrix(X_train, label=y_train, weight=weights)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_valid, label=y_valid)
#     dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=features)
#     dvalid = xgb.DMatrix(X_valid, label=y_valid, feature_names=features)
#     dtrain = xgb.DMatrix(training_data.values, label.values)
    param = {'n_estimators': 4000, 'max_depth': 3, 'min_child_weight': 5, 'gamma': 0, 'subsample': 1.0, 
             'colsample_bytree': 0.8, 'scale_pos_weight':10, 'eta': 0.1, 'silent': 1, 'objective': 'binary:logistic',
             'eval_metric':'auc'}
#     param = {'n_estimators': 4000, 'max_depth': 6, 'seed': 7, 'min_child_weight': 5, 'gamma': 0, 'subsample': 1.0, 
#              'colsample_bytree': 0.8, 'scale_pos_weight': 1, 'eta': 0.09, 'silent': 1, 'objective': 'binary:logistic',
#              'eval_metric':'auc'}
    
    num_round = param['n_estimators']
#     param['nthread'] = 4
    # param['eval_metric'] = "auc"
    plst = param.items()
    evallist = [(dtrain, 'train'), (dvalid, 'eval')]
    # 根本停不下来
#     evallist = [(dvalid, 'eval'), (dtrain, 'train')]
#     evallist = [(dtrain, 'train')]
    bst = xgb.train(plst, dtrain, num_round, evallist, early_stopping_rounds=10)
    bst.save_model('bst.model')
    return bst, features

bst_xgb, features = xgb_model('train_set.csv')

total features:  301
[0]	train-auc:0.913108	eval-auc:0.911621
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 10 rounds.
[1]	train-auc:0.932872	eval-auc:0.930423
[2]	train-auc:0.936241	eval-auc:0.93338
[3]	train-auc:0.938389	eval-auc:0.936325
[4]	train-auc:0.938821	eval-auc:0.937227
[5]	train-auc:0.942359	eval-auc:0.941066
[6]	train-auc:0.943082	eval-auc:0.941563
[7]	train-auc:0.943831	eval-auc:0.942126
[8]	train-auc:0.945939	eval-auc:0.944321
[9]	train-auc:0.946849	eval-auc:0.94524
[10]	train-auc:0.948741	eval-auc:0.946958
[11]	train-auc:0.949038	eval-auc:0.947272
[12]	train-auc:0.950141	eval-auc:0.948423
[13]	train-auc:0.951377	eval-auc:0.949643
[14]	train-auc:0.951717	eval-auc:0.950112
[15]	train-auc:0.952721	eval-auc:0.951315
[16]	train-auc:0.953227	eval-auc:0.952028
[17]	train-auc:0.953574	eval-auc:0.952416
[18]	train-auc:0.954116	eval-auc:0.952997
[19]	train-auc:0.954471	eval-auc:0.953389
[20]	train

In [211]:
print bst_xgb.attributes()

{'best_iteration': '408', 'best_msg': '[408]\ttrain-auc:0.981964\teval-auc:0.976777', 'best_score': '0.976777'}


### 对验证集进行线下测试

In [212]:
def report(pred, label):

    actions = label
    result = pred

    # 所有实际用户商品对
    all_user_item_pair = actions['user_id'].map(str) + '-' + actions['sku_id'].map(str)
    all_user_item_pair = np.array(all_user_item_pair)
    # 所有购买用户
    all_user_set = actions['user_id'].unique()

    # 所有预测购买的用户
    all_user_test_set = result['user_id'].unique()
    all_user_test_item_pair = result['user_id'].map(str) + '-' + result['sku_id'].map(str)
    all_user_test_item_pair = np.array(all_user_test_item_pair)

    # 计算所有用户购买评价指标
    pos, neg = 0,0
    for user_id in all_user_test_set:
        if user_id in all_user_set:
            pos += 1
        else:
            neg += 1
    all_user_acc = 1.0 * pos / ( pos + neg)
    all_user_recall = 1.0 * pos / len(all_user_set)
    print '所有用户中预测购买用户的准确率为 ' + str(all_user_acc)
    print '所有用户中预测购买用户的召回率' + str(all_user_recall)

    pos, neg = 0, 0
    for user_item_pair in all_user_test_item_pair:
        if user_item_pair in all_user_item_pair:
            pos += 1
        else:
            neg += 1
    all_item_acc = 1.0 * pos / ( pos + neg)
    all_item_recall = 1.0 * pos / len(all_user_item_pair)
    print '所有用户中预测购买商品的准确率为 ' + str(all_item_acc)
    print '所有用户中预测购买商品的召回率' + str(all_item_recall)
    F11 = 6.0 * all_user_recall * all_user_acc / (5.0 * all_user_recall + all_user_acc)
    F12 = 5.0 * all_item_acc * all_item_recall / (2.0 * all_item_recall + 3 * all_item_acc)
    score = 0.4 * F11 + 0.6 * F12
    print 'F11=' + str(F11)
    print 'F12=' + str(F12)
    print 'score=' + str(score)
    
    return all_user_acc, all_user_recall, F11, all_item_acc, all_item_recall, F12, score

In [213]:
def validate(valid_set, val_label, model):
    actions = pd.read_csv(valid_set)                #read test_set        
#     users = actions[['user_id', 'sku_id']].copy()
    # 避免预测到非8类商品，所以最后还是再筛一遍的好        
    users = actions[['user_id', 'sku_id', 'cate']].copy()
    
    actions['user_id'] = actions['user_id'].astype(np.int64)
#     test_label= actions[actions['label'] == 1]

#     test_label= actions[(actions['label']==1) & (actions['cate']==8)]
    test_label = pd.read_csv(val_label)
    
    lst_useless = ['brand']
    
    actions.drop(lst_useless, inplace=True, axis=1)
    
#     test_label = test_label[['user_id','sku_id','label']]
    del actions['user_id']
    del actions['sku_id']
    
#     features = list(actions.columns.values)
    
#     del actions['label']
    sub_user_index = users
#     sub_trainning_data = xgb.DMatrix(actions.values, feature_names=features)
    sub_trainning_data = xgb.DMatrix(actions.values)
#     y = model.predict(sub_trainning_data,ntree_limit=model.best_iteration)
    y = model.predict(sub_trainning_data, ntree_limit=model.best_ntree_limit)
    sub_user_index['label'] = y
    
    sub_user_index.to_csv('result_' + valid_set, index=False)
    
#     sub_user_index = sub_user_index[sub_user_index['cate']==8]
#     del sub_user_index['cate']
    rank = 1000
    pred = sub_user_index.sort_values(by='label', ascending=False)[:rank]
#     pred = sub_user_index[sub_user_index['label'] >= 0.05]
    
    print 'No. of raw pred users: ', len(pred['user_id'].unique())
    pred = pred[pred['cate']==8]
    print 'No. of pred users bought cate 8: ', len(pred['user_id'].unique())
    
#     pred = pred[['user_id', 'sku_id']]
    pred = pred[['user_id', 'sku_id', 'label']]
    pred = pred.groupby('user_id').first().reset_index()
    pred['user_id'] = pred['user_id'].astype(int)
    pred['sku_id'] = pred['sku_id'].astype(int)
    
#     print 'No. of pred users after deduplicates: ', len(pred['user_id'].unique())
    true_user = len(test_label['user_id'])   
    pred_ui = len(pred['user_id'].unique())
    print 'pred item: ', len(pred['sku_id'].unique())
    print 'true users: ', true_user
    print 'pred users: ', pred_ui
    test_label['user_id'] = test_label['user_id'].astype(int)
    test_label['sku_id'] = test_label['sku_id'].astype(int)

    all_user_acc, all_user_recall, F11, all_item_acc, all_item_recall, F12, score = report(pred, test_label)   
    
    f_name = 'pred_' + str(rank) + '_' + valid_set
    pred.to_csv(f_name, index=False)
    
    return rank, true_user, pred_ui, all_user_acc, all_user_recall, F11, all_item_acc, all_item_recall, F12, score

# validate('val_set.csv', bst_xgb)

In [214]:
def avg_score():
    rank1, true_user1, pred_ui1, user_acc1, user_recall1, F11_1, item_acc1, item_recall1, F12_1, score1 = validate('val_1.csv', 'label_val_1.csv', bst_xgb)
    print '-------------------------------------------'
    rank2, true_user2, pred_ui2, user_acc2, user_recall2, F11_2, item_acc2, item_recall2, F12_2, score2 = validate('val_2.csv', 'label_val_2.csv', bst_xgb)
    print '-------------------------------------------'
    rank3, true_user3, pred_ui3, user_acc3, user_recall3, F11_3, item_acc3, item_recall3, F12_3, score3 = validate('val_3.csv', 'label_val_3.csv', bst_xgb)
    print '==========================================='
    print 'avg user acc: ', (user_acc1+user_acc2+user_acc3)/3
    print 'avg user recall: ', (user_recall1+user_recall2+user_recall3)/3
    print 'avg item acc: ', (item_acc1+item_acc2+item_acc3)/3
    print 'avg item recall: ', (item_recall1+item_recall2+item_recall3)/3
    print 'avg F11: ', (F11_1+F11_2+F11_3)/3
    print 'avg F12: ', (F12_1+F12_2+F12_3)/3
    print 'avg score: ', (score1+score2+score3)/3
    # make the csv file
    dct_score = {}
    dct_score['rank'] = [rank1, rank2, rank3]
    dct_score['true_user'] = [true_user1, true_user2, true_user3]
    dct_score['pred_ui'] = [pred_ui1, pred_ui2, pred_ui3]
    dct_score['user_acc'] = [user_acc1, user_acc2, user_acc3]
    dct_score['user_recall'] = [user_recall1, user_recall2, user_recall3]
    dct_score['F11'] = [F11_1, F11_2, F11_3]
    dct_score['item_acc'] = [item_acc1, item_acc2, item_acc3]
    dct_score['item_recall'] = [item_recall1, item_recall2, item_recall3]
    dct_score['F12'] = [F12_1, F12_2, F12_3]
    dct_score['score'] = [score1, score2, score3]
    column_order = ['rank', 'true_user', 'pred_ui', 'user_acc', 'user_recall', 'item_acc', 'item_recall', 'F11', 'F12', 
                    'score']
    df_score = pd.DataFrame(dct_score)
    file_name = 'score_' + str(datetime.now().date())[5:] +'_'+ str(rank1) + '.csv'
    df_score[column_order].to_csv(file_name, index=False)
avg_score()

No. of raw pred users:  950
No. of pred users bought cate 8:  950
pred item:  220
true users:  1211
pred users:  950
所有用户中预测购买用户的准确率为 0.147368421053
所有用户中预测购买用户的召回率0.116569525396
所有用户中预测购买商品的准确率为 0.108421052632
所有用户中预测购买商品的召回率0.0850536746491
F11=0.141152747437
F12=0.0930778962588
score=0.11230783673
-------------------------------------------
No. of raw pred users:  950
No. of pred users bought cate 8:  950
pred item:  203
true users:  1259
pred users:  950
所有用户中预测购买用户的准确率为 0.163157894737
所有用户中预测购买用户的召回率0.12360446571
所有用户中预测购买商品的准确率为 0.116842105263
所有用户中预测购买商品的召回率0.0881652104845
F11=0.15489673551
F12=0.0977629029417
score=0.120616435969
-------------------------------------------
No. of raw pred users:  960
No. of pred users bought cate 8:  960
pred item:  219
true users:  1385
pred users:  960
所有用户中预测购买用户的准确率为 0.161458333333
所有用户中预测购买用户的召回率0.11231884058
所有用户中预测购买商品的准确率为 0.120833333333
所有用户中预测购买商品的召回率0.0837545126354
F11=0.150485436893
F12=0.0954732510288
score=0.117478125375
avg user a

### 输出特征重要性

In [27]:
def feature_importance(bst_xgb):
    importance = bst_xgb.get_fscore(fmap=r'xgb.fmap')
    importance = sorted(importance.items(), key=operator.itemgetter(1), reverse=True)

    df = pd.DataFrame(importance, columns=['feature', 'fscore'])
    df['fscore'] = df['fscore'] / df['fscore'].sum()
    file_name = 'feature_importance_' + str(datetime.now().date())[5:] + '.csv'
    df.to_csv(file_name)

feature_importance(bst_xgb)

### 生成提交结果

In [185]:
# sub_file

def submit(pred_set, model):
    actions = pd.read_csv(pred_set)                #read test_set
    
#     print 'total user before: ', len(actions['user_id'].unique())
#     potential = pd.read_csv('potential_user_04-28.csv')
#     lst_user = potential['user_id'].unique().tolist()
#     actions = actions[actions['user_id'].isin(lst_user)]
#     print 'total user after: ', len(actions['user_id'].unique())
    # 提前去掉部分特征
    lst_useless = ['brand']
    
    actions.drop(lst_useless, inplace=True, axis=1)

    
    users = actions[['user_id', 'sku_id', 'cate']].copy()
#     users = actions[['user_id', 'sku_id']].copy()
    
    actions['user_id'] = actions['user_id'].astype(np.int64)
    del actions['user_id']
    del actions['sku_id']
    sub_user_index = users
    sub_trainning_data = xgb.DMatrix(actions.values)
    y = model.predict(sub_trainning_data, ntree_limit=model.best_ntree_limit)
    sub_user_index['label'] = y
    
#     sub_user_index = sub_user_index[sub_user_index['cate']==8]
#     del sub_user_index['cate']
    rank = 1200
    pred = sub_user_index.sort_values(by='label', ascending=False)[:rank]
#     pred = sub_user_index[sub_user_index['label'] >= 0.05]
#     pred = pred[['user_id', 'sku_id', 'label']]
#     pred = pred[pred['label']>0.45]

    print 'No. of raw pred users: ', len(pred['user_id'].unique())
    pred = pred[pred['cate']==8]
    print 'No. of pred users bought cate 8: ', len(pred['user_id'].unique())

    pred = pred[['user_id', 'sku_id']]
#     print 
    pred = pred.groupby('user_id').first().reset_index()
    pred['user_id'] = pred['user_id'].astype(int)
    pred['sku_id'] = pred['sku_id'].astype(int)
    sub_file = 'submission_' + str(rank) + '_' + str(datetime.now().date())[5:] + '.csv'
#     sub_file = 'submission_detail_' + str(datetime.now().date())[5:] + '.csv'
    pred.to_csv(sub_file, index=False, index_label=False)  

submit('test_set.csv', bst_xgb)

No. of raw pred users:  1142
No. of pred users bought cate 8:  1142


### 提交结果除重
除去最后临近三天的发生购买行为的用户商品对

In [186]:
from datetime import datetime

def sub_improv(action, sub_file):
    # 获取4月最近三天的目标用户商品对
    action_4 = pd.read_csv(action)
    action_4['time'] = pd.to_datetime(action_4['time']).apply(lambda x: x.date())
    aim_date = [datetime.strptime(s, '%Y-%m-%d').date() for s in ['2016-04-09', '2016-04-10', '2016-04-11' ,
                                                                  '2016-04-12', '2016-04-13', '2016-04-14', 
                                                                  '2016-04-15']]
    aim_action = action_4[(action_4['type']==4) & (action_4['cate']==8) & (action_4['time'].isin(aim_date))]
    aim_ui = aim_action['user_id'].map(int).map(str) + '-' + aim_action['sku_id'].map(str)
    # 拼接提交数据的用户商品
    sub = pd.read_csv(sub_file)
    before = sub.shape[0]
    sub_ui = sub['user_id'].map(str) + '-' + sub['sku_id'].map(str)
    # 交集
    lst_aim = aim_ui.unique().tolist()
    lst_sub = sub_ui.unique().tolist()
    lst_common = [i for i in lst_aim if i in lst_sub]
    dct_ui = {i.split('-')[0]: i.split('-')[1] for i in lst_common}
    # 从提交结果除掉交集部分
    for k in dct_ui:
        sub.drop(sub[(sub['user_id']==int(k)) & (sub['sku_id']==int(dct_ui[k]))].index, inplace=True)
    print 'No. of records after remove dup: ', sub.shape[0]
    print 'No. of dup: ', before - sub.shape[0]
    if (before - sub.shape[0])!=0:
        file_name = 'submission_' + str(datetime.now().date())[5:] + '_improv.csv'
        sub.to_csv(file_name, index=False, index_label=False)
    
sub_improv('data/JData_Action_201604.csv', 'submission_1200_05-25.csv')

No. of records after remove dup:  1142
No. of dup:  0
